## Routing agent with OCI GenAI and LangGraph

Implements an example of agent with router and final aggregator using **LangGraph** and **OCI GenAI**

In [1]:
# Schema for structured output
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display, Markdown

from oci_models import create_model_for_routing
from prompts_library import PROMPT_ROUTER
from utils import get_console_logger

In [2]:
DEBUG = False

logger = get_console_logger()

# this is the model for the router
# we're using Command-r-plus
llm_router = create_model_for_routing()

In [3]:
#
# tools definition
#
# State: the input to the agent
class State(TypedDict):
    """
    Defines the internal state of the agent
    """

    input: str
    decision: str
    output: str

In [4]:
#
# json schema for the output of the Router
#
json_route = {
    "title": "Route",
    "description": "Defines the output of the routing logic",
    "type": "object",
    "properties": {
        "step": {
            "description": "The next step in the routing process",
            "type": "string",
            "enum": [
                "generate_sql",
                "not_allowed",
                "analyze_data",
                "answer_directly",
                "not_defined",
            ],
        }
    },
    "required": ["step"],
}

# Augment the LLM with schema for structured output
router = llm_router.with_structured_output(json_route)

### define the router

In [5]:
def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    logger.info("Called router...")

    if DEBUG:
        logger.info("Input: %s", state["input"])

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            # with sys_msg have troubles
            # this doesn't work SystemMessage(PROMPT_ROUTER),
            HumanMessage(content=PROMPT_ROUTER + state["input"]),
        ]
    )

    if DEBUG:
        logger.info("Decision: %s", decision)

    return {"decision": decision["step"]}

## let's test the router

In [6]:
# test the router
def test_router(query: str):
    # set the input
    state_test = State(input=query)

    outcome = llm_call_router(state_test)

    return outcome


queries = [
    "analyze the data and create a report",
    "I want all the orders made in Italy in 2024",
    "Who is Enrico Fermi",
    "drop table sales",
]

for query in queries:
    try:
        print("")
        outcome = test_router(query)

        print("The request is: ", query)
        print(outcome)
    except Exception as e:
        print(e)

2025-02-07 17:17:40,405 - Called router...


2025-02-07 17:17:41,769 - Called router...


The request is:  analyze the data and create a report
{'decision': 'analyze_data'}



2025-02-07 17:17:42,802 - Called router...


The request is:  I want all the orders made in Italy in 2024
{'decision': 'generate_sql'}



2025-02-07 17:17:43,828 - Called router...


The request is:  Who is Enrico Fermi
{'decision': 'answer_directly'}

The request is:  drop table sales
{'decision': 'not_allowed'}


#### in another notebook we wil show how to plug this router in a LangGraph agent